In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#remove finals
#make rounds R1 into 1 etc.
stats = pd.read_csv("stats_with_ratings.csv").drop("Unnamed: 0", axis = 1)
finals = ["GF", "PF", "QF", "EF", "SF"]
stats = stats[~stats.Round.isin(finals)]
stats["Round"] = stats["Round"].apply(lambda x: int(x[1:]))
stats = stats.sort_values(["Team", "Season", "Round"])

In [3]:
r12012 = stats[stats.Season == 2012]
r12012 = r12012[r12012.Round == 1]

In [4]:
teams = {i for i in r12012["Team"]}
seperated = [r12012[r12012.Team == i].reset_index(drop = True) for i in teams]

In [5]:
#Seperate into a list of lists, with each list being the dataframes for the teams in a certain match
pairings = []
done = []

for i in seperated:
    team = i.loc[0, "Team"]
    if team not in done:
        pairings.append(i)
        done.append(i.Team[0])
        done.append(i.Opposition[0])
        opp = i.Opposition[0]
        for j in seperated:
            if j.Team[0] == opp:
                pairings.append(j)
                
pairings = [pairings[i:i + 2] for i in range(0, len(pairings), 2)]
pairings = [pd.concat(i).sort_values("Ratings", ascending = False).reset_index(drop = True) for i in pairings]

In [6]:
#Tallying votes for r1 2012
from collections import defaultdict
votes = defaultdict(int)
for i in pairings:
    best = i.loc[0, "Player"]
    second_best = i.loc[1, "Player"]
    third_best = i.loc[2, "Player"]
    votes[best] += 3
    votes[second_best] += 2
    votes[third_best] += 1
    

In [7]:
rounds = range(1,24)
seasons = range(2012, 2019)

votes_by_season = dict()



for s in seasons:
    season = stats[stats.Season == s]
    votes = defaultdict(int)
    for r in rounds:
        
        round_ = season[season.Round == r]
        teams = {i for i in round_["Team"]}
        seperated = [round_[round_.Team == i].reset_index(drop = True) for i in teams]
        
        pairings = []
        done = []

        for i in seperated:
            team = i.loc[0, "Team"]
            if team not in done:
                pairings.append(i)
                done.append(i.Team[0])
                done.append(i.Opposition[0])
                opp = i.Opposition[0]
                for j in seperated:
                    if j.Team[0] == opp:
                        pairings.append(j)
                        
        pairings = [pairings[i:i + 2] for i in range(0, len(pairings), 2)]
        pairings = [pd.concat(i).sort_values("Ratings", ascending = False).reset_index(drop = True) for i in pairings]
        
        for i in pairings:
            best = i.loc[0, "Player"]
            second_best = i.loc[1, "Player"]
            third_best = i.loc[2, "Player"]
            votes[best] += 3
            votes[second_best] += 2
            votes[third_best] += 1
    
    votes_by_season[s] = votes

In [8]:
{k: v for k, v in sorted(votes_by_season[2018].items(), key=lambda item: item[1], reverse = True)}

{'Mitchell, Tom': 36,
 'Macrae, Jack': 29,
 'Gaff, Andrew': 29,
 'Ross, Sebastian': 28,
 'Beams, Dayne': 26,
 'Lloyd, Jake': 23,
 'Sidebottom, Steele': 22,
 'Ablett, Gary': 20,
 'Dangerfield, Patrick': 20,
 'Higgins, Shaun': 20,
 'Heppell, Dyson': 19,
 'Martin, Dustin': 19,
 'Fyfe, Nat': 19,
 'Crouch, Matt': 18,
 'Laird, Rory': 18,
 'Coniglio, Stephen': 17,
 'Steven, Jack': 17,
 'Whitfield, Lachie': 17,
 'Zaharakis, David': 17,
 'Brayshaw, Angus': 17,
 'Cripps, Patrick': 16,
 'Oliver, Clayton': 16,
 'Langdon, Ed': 16,
 'Franklin, Lance': 15,
 'Kennedy, Josh P': 15,
 'Hunter, Lachie': 15,
 'Yeo, Elliot': 15,
 'Kelly, Josh': 15,
 'Shiels, Liam': 14,
 'Treloar, Adam': 14,
 'Selwood, Joel': 13,
 'Neale, Lachie': 13,
 'Simpson, Kade': 13,
 'Phillips, Tom': 13,
 'Gibbs, Bryce': 12,
 'Wines, Ollie': 12,
 'Billings, Jack': 11,
 'Polec, Jared': 11,
 'Witherden, Alex': 11,
 'Merrett, Zach': 11,
 'Redden, Jack': 11,
 'Duncan, Mitch': 10,
 'Swallow, David': 8,
 'Gunston, Jack': 8,
 'Hogan, Jesse':

In [9]:
# Finding actual Brownlow votes from those years from the dataset
actual_votes_by_season = dict()
for i in seasons:
    actual_votes = dict()
    players = stats[stats.Season == i].groupby("Player")
    players = players.sum().sort_values("BrownlowVotes", ascending = False)
    for j in players.index:
        if not players.loc[j, "BrownlowVotes"]:
            break
        actual_votes[j] = players.loc[j, "BrownlowVotes"]
    actual_votes_by_season[i] = (actual_votes)

In [10]:
b = actual_votes_by_season[2018]

In [11]:
a = {k: v for k, v in sorted(votes_by_season[2018].items(), key=lambda item: item[1], reverse = True)}

In [12]:
sd = 0
for i in a.keys()&b.keys():
    sd += (a[i] - b[i])**2

In [13]:
c = [(k,v) for k,v in a.items()]
d = [(k,v) for k,v in b.items()]

In [14]:
corrects = []
almost = []
for i in seasons:
    correct = 0
    a = {k: v for k, v in sorted(votes_by_season[i].items(), key=lambda item: item[1], reverse = True)}
    b = b = actual_votes_by_season[i]
    c = [(k,v) for k,v in a.items()]
    d = [(k,v) for k,v in b.items()]
    for j in range(5):
        if c[j][0] == d[j][0]:# or c[j+1][0] == d[j][0] or c[j-1][0] == d[j][0] or c[j+2][0] == d[j][0] or c[j-2][0] == d[j][0]:
            correct += 1
            almost.append(c[j][0])
    almost.append("YEAR")
    corrects.append(correct)
    
        

In [15]:
sum(corrects)/(7*5) 

0.2

In [16]:
sorted(votes_by_season[2014].items(), reverse = True, key=lambda x: (x[1], x[0]))

[('Macrae, Jack', 26),
 ('Rockliff, Tom', 20),
 ('Johnson, Steve', 20),
 ('Boak, Travis', 20),
 ('Tyson, Dom', 19),
 ('Sidebottom, Steele', 19),
 ('Ablett, Gary', 19),
 ('Greenwood, Levi', 18),
 ('Martin, Dustin', 17),
 ('Lewis, Jordan', 17),
 ('Gray, Robbie', 17),
 ('Boyd, Matthew', 17),
 ('Beams, Dayne', 17),
 ('Zaharakis, David', 16),
 ('Selwood, Joel', 16),
 ('Gibbs, Bryce', 16),
 ('Heppell, Dyson', 15),
 ('Harvey, Brent', 15),
 ('Deledio, Brett', 15),
 ('Prestia, Dion', 14),
 ('Kennedy, Josh P', 14),
 ('Goddard, Brendon', 14),
 ('Gibson, Sam', 14),
 ('Vince, Bernie', 13),
 ('Treloar, Adam', 13),
 ('Fyfe, Nat', 13),
 ('Barlow, Michael', 13),
 ('Stanton, Brent', 12),
 ('Smith, Brodie', 12),
 ('McVeigh, Jarrad', 12),
 ('Jones, Nathan', 12),
 ('Cotchin, Trent', 12),
 ('Sloane, Rory', 11),
 ('Pendlebury, Scott', 11),
 ('Murphy, Marc', 11),
 ('Hanley, Pearce', 11),
 ('Ellis, Brandon', 11),
 ('Dangerfield, Patrick', 11),
 ('Thompson, Scott', 10),
 ('Stokes, Mathew', 10),
 ('Simpson, Kade